<a href="https://colab.research.google.com/github/hackohackob/AngularJS-learning/blob/master/accesible_gaussian_splatting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gaussian Splatting - Accessible 3D Reconstruction

This notebook adapts the [original](https://github.com/graphdeco-inria/gaussian-splatting) reference implementation of "3D Gaussian Splatting for Real-Time Radiance Field Rendering" to create a simplified, educational and fully automated pipeline with a graphical interface for non-expert users.  
Developed by Daniel Cob Beirute, Instituto Tecnológico de Costa Rica (2025).

# Dependencies installation

In [1]:
# Clone and install Guassian Splatting repository
%cd /content
!git clone --recursive https://github.com/DanielCob/accesible-gaussian-splatting.git
!pip install -q plyfile

%cd /content/accesible-gaussian-splatting
!pip install -q /content/accesible-gaussian-splatting/submodules/diff-gaussian-rasterization
!pip install -q /content/accesible-gaussian-splatting/submodules/simple-knn

/content
Cloning into 'accesible-gaussian-splatting'...
remote: Enumerating objects: 885, done.
remote: Total 885 (delta 0), reused 0 (delta 0), pack-reused 885 (from 1)
Receiving objects: 100% (885/885), 78.72 MiB | 33.06 MiB/s, done.
Resolving deltas: 100% (479/479), done.
Submodule 'SIBR_viewers' (https://gitlab.inria.fr/sibr/sibr_core.git) registered for path 'SIBR_viewers'
Submodule 'submodules/diff-gaussian-rasterization' (https://github.com/graphdeco-inria/diff-gaussian-rasterization.git) registered for path 'submodules/diff-gaussian-rasterization'
Submodule 'submodules/fused-ssim' (https://github.com/rahul-goel/fused-ssim.git) registered for path 'submodules/fused-ssim'
Submodule 'submodules/simple-knn' (https://gitlab.inria.fr/bkerbl/simple-knn.git) registered for path 'submodules/simple-knn'
Cloning into '/content/accesible-gaussian-splatting/SIBR_viewers'...
remote: Enumerating objects: 3293, done.        
remote: Counting objects: 100% (322/322), done.        
remote: Compr

In [3]:
# Install dependencies
!sudo apt-get install -y colmap
!sudo apt-get install -y imagemagick
!sudo apt-get install -y xvfb # OpenGL workaround for Google Colab

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  at-spi2-core gsettings-desktop-schemas libamd2 libatk-bridge2.0-0
  libatk1.0-0 libatk1.0-data libatspi2.0-0 libcamd2 libccolamd2 libceres2
  libcholmod3 libcolamd2 libcxsparse3 libdouble-conversion3 libevdev2
  libfreeimage3 libgflags2.2 libglew2.2 libgoogle-glog0v5 libgtk-3-0
  libgtk-3-bin libgtk-3-common libgudev-1.0-0 libilmbase25 libinput-bin
  libinput10 libjxr0 libmd4c0 libmetis5 libmtdev1 libopenexr25 libqt5core5a
  libqt5dbus5 libqt5gui5 libqt5network5 libqt5svg5 libqt5widgets5 libraw20
  librsvg2-common libspqr2 libsuitesparseconfig5 libwacom-bin libwacom-common
  libwacom9 libxcb-icccm4 libxcb-image0 libxcb-keysyms1 libxcb-render-util0
  libxcb-util1 libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1 libxcomposite1
  libxkbcommon-x11-0 libxtst6 qt5-gtk-platformtheme qttranslations5-l10n
  session-migration
Suggested packages:
  gle

# Data upload point
(upload a short video orbiting a object or scene)

In [4]:
from google.colab import files
import os

uploaded = files.upload()
# The user uploads a single video file
uploaded_filename = list(uploaded.keys())[0]

# Create the input directory if it doesn't exist
input_dir = '/content/accesible-gaussian-splatting/data/video'
os.makedirs(input_dir, exist_ok=True)

# Define the new path for the video
video_path = os.path.join(input_dir, uploaded_filename)

# Move the uploaded video to the input directory
os.rename(uploaded_filename, video_path)

print("Video uploaded and moved to:", video_path)

Saving 2026-02-07 03-24-38.mov to 2026-02-07 03-24-38.mov
Video uploaded and moved to: /content/accesible-gaussian-splatting/data/video/2026-02-07 03-24-38.mov


# Preproccesing

In [5]:
# Create a directory to store the extracted frames
!mkdir /content/accesible-gaussian-splatting/data/input

# Use ffmpeg to extract frames from the uploaded video
!ffmpeg -i "{video_path}" -qscale:v 1 -qmin 1 -vf fps=2 /content/accesible-gaussian-splatting/data/input/%04d.jpg

print("Frames extracted to /content/gaussian-splatting/data/input/")

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [10]:
!nvidia-smi

Sat Feb  7 01:43:07 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [11]:
!ps aux --sort=-%cpu | head -n 20

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root        5355  0.8  1.5 1725544 209064 ?      Sl   01:22   0:10 node /datalab/web/pyright/pyright-langserver.js --stdio --cancellationReceive=file:9d8152226d176377dd23429bc8fc4526499234e10d
root          65  0.6  0.0      0     0 ?        Z    01:03   0:16 [python3] <defunct>
root        2647  0.6  0.9 692412 130988 ?       Ssl  01:13   0:11 /usr/bin/python3 -m colab_kernel_launcher -f /root/.local/share/jupyter/runtime/kernel-962bd518-af4d-4814-a1c9-18c205fcacb3.json
root          66  0.2  0.6 103340 80492 ?        S    01:03   0:07 python3 /usr/local/bin/colab-fileshim.py
root         111  0.2  1.1 500584 153196 ?       Sl   01:03   0:07 /usr/bin/python3 /usr/local/bin/jupyter-server --debug --transport="ipc" --ip=172.28.0.12 --ServerApp.token= --port=9000 --FileContentsManager.root_dir=/ --FileContentsManager.allow_hidden=True --ServerApp.log_format="|%(levelname)s|%(message)s" --ServerApp.iopub_data_rate_

In [17]:
!python3 resize.py --source_path /content/accesible-gaussian-splatting/data --max_side 1600

Found 125 images in /content/accesible-gaussian-splatting/data/input
Resized: 125


In [22]:
!colmap -h | grep -i cuda -n || true
!colmap feature_extractor -h | grep -i use_gpu -n || true

2:              (Commit Unknown on Unknown without CUDA)
27:  --SiftExtraction.use_gpu arg (=1)


In [24]:
%cd /content/accesible-gaussian-splatting
!python convert.py -s /content/accesible-gaussian-splatting/data/ --resize

/content/accesible-gaussian-splatting
ERROR: Failed to parse options - the argument ('1--siftextraction.max_num_features') for option '--SiftExtraction.use_gpu' is invalid. Valid choices are 'on|off', 'yes|no', '1|0' and 'true|false'.
ERROR:root:Feature extraction failed with code 256. Exiting.


# Training

In [ ]:
%cd /content/accesible-gaussian-splatting
!python train.py -s /content/accesible-gaussian-splatting/data/

# Training with Evaluation (Train/Test split)

In [ ]:
%cd /content/accesible-gaussian-splatting
!python train.py -s /content/accesible-gaussian-splatting/data/ --eval
!python render.py -m /content/accesible-gaussian-splatting/data/
!python metrics.py -m /content/accesible-gaussian-splatting/data/

# Exporting output
(.zip can later be visualized using other visualization tools like the one provided by [Camenduru](https://colab.research.google.com/github/camenduru/gaussian-splatting-colab/blob/main/gaussian_splatting_viewer_colab.ipynb) )

In [ ]:
import shutil
import os
from google.colab import files

output_dir = '/content/accesible-gaussian-splatting/output'
compressed_output_path = '/content/accesible-gaussian-splatting/output.zip'

# Compress the output folder
shutil.make_archive(compressed_output_path.replace('.zip', ''), 'zip', output_dir)

# Provide a download link
files.download(compressed_output_path)

print(f"Output folder compressed to {compressed_output_path} and ready for download.")